In [65]:
# importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [66]:
''' Scraping APSRTC bus details from RedBus website '''

# Function to setup the driver
def start_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Function to load a webpage
def open_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to get bus routes (links and names)
def get_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    bus_links = [route.get_attribute('href') for route in routes]
    bus_names = [route.text.strip() for route in routes]
    return bus_links, bus_names

# Function to extract bus details from a specific route page
def extract_bus_details(driver, route_url, route_name):
    bus_info = []
    try:
        driver.get(route_url)
        time.sleep(5)  # Wait for the page to load
        
        try:
            # Find and click "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load
            
            # Scroll to load all bus information
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load
            
            # Scrape bus information
            bus_names = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_types = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            depart_times = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            durations = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reach_times = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            ratings = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            prices = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_avail = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")
            
            for i in range(len(bus_names)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_link": route_url,
                    "Bus_Name": bus_names[i].text,
                    "Bus_Type": bus_types[i].text,
                    "Departing_Time": depart_times[i].text,
                    "Duration": durations[i].text,
                    "Reaching_Time": reach_times[i].text,
                    "Star_Rating": ratings[i].text if i < len(ratings) else '0',
                    "Price": prices[i].text,
                    "Seat_Availability": seat_avail[i].text if i < len(seat_avail) else '0'
                }
                bus_info.append(bus_detail)
        except Exception as e:
            print(f"Error scraping details from {route_url}: {e}")
    except Exception as e:
        print(f"Error loading page {route_url}: {e}")
    
    return bus_info

# Function to scrape all pages and extract bus details
def scrape_all_bus_data(driver, base_url, num_pages=2):
    all_bus_data = []

    for page in range(1,num_pages+1):
        try:
            # Open the base URL and handle pagination if necessary
            open_page(driver, base_url)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            # Get bus routes on the current page
            bus_links, bus_names = get_bus_routes(driver)
            
            # Iterate over bus routes using an index
            for i in range(len(bus_links)):
                link = bus_links[i]
                name = bus_names[i]
                
                # Extract bus details for each route
                bus_details = extract_bus_details(driver, link, name)
                for bus_detail in bus_details:
                    all_bus_data.append(bus_detail)
        except Exception as e:
            print(f"Error scraping page {page}: {e}")
    
    return all_bus_data


driver = start_driver()  # Start the browser
base_url = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_A = pd.DataFrame(bus_data)
df_A  # Display the DataFrame or use it as needed

# Close the browser
driver.quit()


In [67]:
# Convert bus data to a DataFrame
df_A = pd.DataFrame(bus_data)
df_A  # Display the DataFrame of APSRTC bus details


,Route_Name,Route_link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),23:10,07h 20m,06:30,4.6,400,24 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,Scania AC Multi Axle Sleeper (2+1),23:00,07h 30m,06:30,4.5,INR 639,11 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,AC Sleeper (2+1),22:40,08h 05m,06:45,4.6,INR 599,12 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdIntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),22:45,06h 50m,05:35,4.3,INR 419,18 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Svkdt travels,A/C Sleeper (2+1),22:20,07h 30m,05:50,4.4,INR 589,12 Seats available
...,...,...,...,...,...,...,...,...,...,...
924,Bangalore to Rayachoti,https://www.redbus.in/bus-tickets/bangalore-to...,Abhishek Bus,A/C Sleeper (2+1),21:00,06h 10m,03:10,2.5,INR 749,22 Seats available
925,Bangalore to Rayachoti,https://www.redbus.in/bus-tickets/bangalore-to...,Sri Shrinivasa Bus Transport (SSBT),Non A/C Seater / Sleeper (2+1),21:00,05h 00m,02:00,2.4,INR 490,29 Seats available
926,Bangalore to Rayachoti,https://www.redbus.in/bus-tickets/bangalore-to...,Sai Balaji Travels,A/C Sleeper (2+1),20:00,05h 15m,01:15,3.7,INR 1999,34 Seats available
927,Bangalore to Rayachoti,https://www.redbus.in/bus-tickets/bangalore-to...,BABA TRAVELS,Non A/C Seater / Sleeper (2+1),20:30,05h 15m,01:45,3.6,INR 2000,35 Seats available


In [68]:
#converting dataframes to csv
df_A.to_csv('APSRTC_bus_details.csv', index=False)


In [ ]:
'''Scraping bus details of TSRTC from Redbus'''
driver = start_driver()  # Start the browser
base_url = ("https://www.redbus.in/online-booking/tsrtc?utm_source=rtchometile") #url for TSRTC buses
                      #number of pages to scrape                  
# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_T = pd.DataFrame(bus_data) 

# Close the browser
driver.quit()

In [71]:
# Convert bus data to a DataFrame
df_T = pd.DataFrame(bus_data)
df_T #display the data frame of TSRTC buses

,Route_Name,Route_link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),23:10,07h 20m,06:30,4.6,400,24 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,Scania AC Multi Axle Sleeper (2+1),23:00,07h 40m,06:40,4.5,INR 639,11 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,AC Sleeper (2+1),22:40,08h 15m,06:55,4.6,INR 589,12 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdIntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),22:45,06h 50m,05:35,4.3,INR 419,18 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Svkdt travels,A/C Sleeper (2+1),22:20,07h 30m,05:50,4.4,INR 589,12 Seats available
...,...,...,...,...,...,...,...,...,...,...
583,Hyderabad to Bhadrachalam,https://www.redbus.in/bus-tickets/hyderabad-to...,Morning Star Travels,A/C Sleeper (2+1),21:30,07h 40m,05:10,2.2,INR 600,21 Seats available
584,Hyderabad to Bhadrachalam,https://www.redbus.in/bus-tickets/hyderabad-to...,MAHABAHUSRIKVRTRAVELS,Non A/C Seater / Sleeper (2+1),19:00,04h 50m,23:50,1.6,INR 750,34 Seats available
585,Hyderabad to Bhadrachalam,https://www.redbus.in/bus-tickets/hyderabad-to...,Gupta Travels Galaxy,NON A/C Seater / Sleeper (2+2),21:00,07h 00m,04:00,1.8,720,20 Seats available
586,Hyderabad to Bhadrachalam,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Karan Sai Tours and Travels,Volvo Multi-Axle A/C seater/sleeper (2+1),22:15,06h 45m,05:00,2.9,INR 5000,47 Seats available


In [72]:
df_T.to_csv('TSRTC_bus_details.csv', index=False) #converting dataframes to csv

In [73]:
'''Scraping bus detais of UPSRTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url = ("https://www.redbus.in/online-booking/upsrtc?utm_source=rtchometile") #url for UPSRTC buses
                

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

#Convert bus data to data frame
df_U= pd.DataFrame(bus_data)
                      
                      



In [74]:
df_U #display the data frame of UPSRTC buses

,Route_Name,Route_link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - HDD0069,Ordinary Non AC Seater 2+3,19:29,04h 11m,23:40,3.3,INR 450,52 Seats available
1,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - HDD0031,Ordinary Non AC Seater 2+3,19:45,07h 25m,03:10,3.3,INR 450,52 Seats available
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - KSB0089,Shatabdi AC Seater 2+2,19:46,05h 45m,01:31,3.3,INR 598,37 Seats available
3,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - HDD0105,Rajdhani Non AC Seater 2+3,20:00,05h 41m,01:41,3.3,INR 450,51 Seats available
4,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - AWD0240,Janrath AC Seater 2+3,20:02,06h 13m,02:15,3.3,INR 520,48 Seats available
...,...,...,...,...,...,...,...,...,...,...
477,Lucknow to Varanasi,https://www.redbus.in/bus-tickets/lucknow-to-v...,Mahalaxmi Travels,Bharat Benz A/C Seater /Sleeper (2+1),23:15,06h 30m,05:45,3.5,INR 381,20 Seats available
478,Lucknow to Varanasi,https://www.redbus.in/bus-tickets/lucknow-to-v...,Yolo Bus,A/C Seater / Sleeper (2+1),22:19,07h 24m,05:43,3.9,INR 399,25 Seats available
479,Lucknow to Varanasi,https://www.redbus.in/bus-tickets/lucknow-to-v...,Goyal Travels,A/C Seater / Sleeper (2+1),23:49,06h 31m,06:20,1.8,INR 500,28 Seats available
480,Agra to Bareilly,https://www.redbus.in/bus-tickets/agra-to-bare...,UPSRTC - BLY0762,Janrath AC Seater 2+3,22:06,05h 54m,04:00,3.3,INR 417,46 Seats available


In [75]:
df_U.to_csv('UPSRTC_bus_details.csv', index=False) #converting dataframes to csv

In [ ]:
'''Scraping  the bus details of South Bengal from Redbus website'''
driver = start_driver()  # Start the browser
base_url = ('https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile') 
#url for South Bengal buses 

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_SB = pd.DataFrame(bus_data)

In [77]:
df_SB #display the data frame of South Bengal buses

,Route_Name,Route_link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,Express Line,Volvo A/C Seater (2+2),18:30,04h 20m,22:50,4.5,INR 486,16 Seats available
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KARUNAMOYEE - BURDWAN - A/C - 18:45 (BWN...,AC Seater (2+2),18:45,02h 00m,20:45,2.9,INR 200,29 Seats available
2,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KOLKATA - BURDWAN - A/C - 18:45 (BWN DEP...,AC Seater (2+2),18:45,02h 00m,20:45,2.8,INR 180,37 Seats available
3,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KARUNAMOYEE - BURDWAN - 19:30 (BWN DEPOT...,Non AC Seater (2+3),19:30,02h 00m,21:30,3.6,INR 100,51 Seats available
4,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KOLKATA - BURDWAN - 20:00 (BWN DEPOT) - 103,Non AC Seater (2+3),20:00,02h 00m,22:00,3.6,INR 95,52 Seats available
...,...,...,...,...,...,...,...,...,...,...
113,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,Basu Travels,A/C Seater Push Back (2+3),23:30,02h 15m,01:45,2.0,INR 259,20 Seats available
114,Kolkata to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/kolkata-to-n...,Basu Travels,A/C Seater Push Back (2+3),23:55,02h 20m,02:15,3.0,INR 469,21 Seats available
115,Kolkata to Mecheda (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-m...,SBSTC-GARIA - DIGHA - 22:30 (DIGHA DEPOT) - 3745,Non AC Seater (2+3),23:45,00h 15m,00:00,4.0,INR 68,36 Seats available
116,Digha to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/digha-to-dur...,SBSTC-DIGHA - DURGAPUR - VIA - BISHNUPUR - 2/2...,Non AC Seater (2+2),20:30,08h 10m,04:40,3.7,INR 233,9 Seats available


In [78]:
df_SB.to_csv('South_Bengal_bus_details.csv', index=False) #converting dataframes to csv

In [ ]:
'''Scraping bus details of HRTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url = ("https://www.redbus.in/online-booking/hrtc?utm_source=rtchometile") #url for HRTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_H = pd.DataFrame(bus_data)


In [80]:
df_H #display the data frame of HRTC buses

,Route_Name,Route_link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 10m,04:45,4.0,INR 912,5 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 591,A/C Executive (2+3),21:00,10h 20m,07:20,3.7,INR 632,37 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 8,Himsuta AC Seater Volvo/Scania 2+2,21:02,09h 28m,06:30,4.7,INR 912,24 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 570,Ordinary 3+2 Non AC Seater,21:25,09h 20m,06:45,2.5,INR 512,37 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 20,Himsuta AC Seater Volvo/Scania 2+2,21:36,07h 54m,05:30,4.1,INR 887,29 Seats available
...,...,...,...,...,...,...,...,...,...,...
262,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 1213,Ordinary 3+2 Non AC Seater,22:47,06h 43m,05:30,4.8,INR 410,33 Seats available
263,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,VE A/C Semi Sleeper (2+2),23:50,05h 00m,04:50,4.8,664,20 Seats available
264,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),23:55,05h 15m,05:10,4.4,617,10 Seats available
265,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,Scania Multi-Axle AC Semi Sleeper (2+2),23:55,05h 15m,05:10,4.5,569,2 Seats available


In [81]:
df_H .to_csv('HRTC_bus_details.csv', index=False) #converting dataframes to csv

In [ ]:
'''Scraping bus details of WBTC from Redbus website '''
driver = start_driver()  # Start the browser
base_url=("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile") #url of WBTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_W = pd.DataFrame(bus_data)

In [83]:
df_W #display the data frame of WBTC buses

,Route_Name,Route_link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Aradhana Travels,A/C Seater / Sleeper (2+2),18:35,05h 20m,23:55,1.1,450,3 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Satya Paribahan,A/C Seater (2+3),23:00,06h 10m,05:10,2.5,INR 420,18 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Aradhana Bus Service,A/C Seater / Sleeper (2+2),23:00,05h 30m,04:30,2.1,INR 550,26 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,The Joy Ride,A/C Seater / Sleeper (2+2),21:15,06h 20m,03:35,5.0,INR 650,11 Seats available
4,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,Express Line,Volvo A/C Seater (2+2),18:35,04h 15m,22:50,4.5,INR 486,14 Seats available
...,...,...,...,...,...,...,...,...,...,...
62,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...,Friends Travels,A/C Seater (2+2),23:45,05h 45m,05:30,1.0,INR 350,24 Seats available
63,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...,Snemita Parisheba,AC Seater (2+3),19:05,04h 40m,23:45,1.0,INR 370,20 Seats available
64,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...,Anadi Travels (Disha Gold),A/C Seater Push Back (2+3),23:00,05h 40m,04:40,1.0,INR 500,60 Seats available
65,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...,Ankita Paribahan(Atithi),A/C Seater Push Back (2+3),22:40,06h 40m,05:20,0,399,20 Seats available


In [84]:
df_W .to_csv('WBTC_bus_details.csv', index=False) #converting dataframes to csv

In [85]:
'''Scraping  bus details of  RSRTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url=('https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile') #url of RSRTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_R = pd.DataFrame(bus_data)

In [86]:
df_R #display the data frame of RSRTC buses

,Route_Name,Route_link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 189622,Express Non AC Seater 2+3,22:00,07h 00m,05:00,3.5,INR 294,42 Seats available
1,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,Jain travels regd,A/C Seater / Sleeper (2+1),22:15,06h 15m,04:30,4.1,INR 460,32 Seats available
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,Kalpana Travels,NON A/C Sleeper (2+1),22:00,06h 54m,04:54,4.1,INR 450,17 Seats available
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,Ravi Travels,Non A/C Seater / Sleeper (2+1),21:30,06h 45m,04:15,5.0,INR 400,23 Seats available
4,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,R.S.Sodha Travels,A/C Seater / Sleeper (2+1),21:00,06h 10m,03:10,5.0,648,26 Seats available
...,...,...,...,...,...,...,...,...,...,...
535,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Gagan Sarkar Upkar Travels,NON A/C Sleeper (2+1),22:15,04h 00m,02:15,2.3,INR 600,27 Seats available
536,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Jai Shree Ganesh Travels,AC Sleeper (2+1),19:45,04h 45m,00:30,1.0,INR 2380.95,35 Seats available
537,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Intercity Travels Indore,NON A/C Sleeper (2+1),19:15,03h 45m,23:00,1.5,INR 600,10 Seats available
538,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Ganesh Travels,A/C Seater / Sleeper (2+1),20:55,04h 50m,01:45,2.9,INR 1300,10 Seats available


In [87]:
df_R .to_csv('RSRTC_bus_details.csv', index=False) #converting dataframes to csv

In [88]:
'''Scraping bus details of KSRTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url = ("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile") #url of KSRTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_K = pd.DataFrame(bus_data)

In [89]:
df_K #display the data frame of KSRTC buses

,Route_Name,Route_link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 789,Super Fast Non AC Seater (2+3),20:00,07h 45m,03:45,3.9,INR 473,2 Seats available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1586,Super Express Non AC Seater Air Bus (2+2),21:30,09h 25m,06:55,3.6,INR 640,9 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 2133,Swift Deluxe Non AC Air Bus (2+2),22:10,09h 25m,07:35,4.1,INR 640,3 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 2063,SWIFT-GARUDA A/C SEATER BUS,22:30,08h 00m,06:30,3.7,INR 627,1 Seat available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 574,Super Express Non AC Seater Air Bus (2+2),23:00,10h 40m,09:40,3.2,INR 469,4 Seats available
...,...,...,...,...,...,...,...,...,...,...
352,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,KSRTC (Kerala) - 2690,Super Fast Non AC Seater (2+3),23:06,06h 24m,05:30,2.6,INR 332,46 Seats available
353,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Sonia Travels,A/C Seater/Sleeper (2+1),19:10,07h 19m,02:29,3.0,INR 750,19 Seats available
354,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,A1 Travels,A/C Seater / Sleeper (2+1),23:59,07h 01m,07:00,2.5,INR 650,30 Seats available
355,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Madhavi Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),23:59,04h 31m,04:30,4.7,759,33 Seats available


In [90]:
df_K .to_csv('KSRTC_bus_details.csv', index=False) #converting dataframes to csv

In [ ]:
'''Scraping bus details of ASTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url=('https://www.redbus.in/online-booking/astc') #url of ASTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_AS = pd.DataFrame(bus_data)


In [94]:
df_AS #display the data frame of ASTC buses

,Route_Name,Route_link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.0,INR 400,1 Seat available
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Himalayan Travels,NON AC Seater / Sleeper 2+1,23:40,04h 20m,04:00,2.1,INR 500,14 Seats available
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:45,05h 30m,03:15,3.8,INR 550,8 Seats available
3,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Shopuris Travels,Non AC Seater (2+1),23:00,05h 00m,04:00,0,INR 550,25 Seats available
4,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:00,04h 00m,00:00,3.5,350,19 Seats available
5,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:30,04h 30m,01:00,4.2,350,31 Seats available
6,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),A/C Seater / Sleeper (2+1),20:45,05h 45m,02:30,3.3,INR 500,31 Seats available
7,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),21:15,04h 30m,01:45,3.2,INR 450,11 Seats available
8,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),21:45,04h 15m,02:00,3.0,INR 450,11 Seats available
9,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),20:00,05h 15m,01:15,2.9,400,19 Seats available


In [95]:
df_AS.to_csv("ASTC_bus_detais.csv",index=False)  #converting dataframes to csv

In [ ]:
'''Scraping Bus details of BSRTC from Redbus website'''
driver = start_driver()  # Start the browser
base_url=('https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile') #url of BSRTC buses

# Scrape bus data across multiple pages
bus_data = scrape_all_bus_data(driver, base_url,num_pages=2)

# Convert bus data to a DataFrame
df_BS = pd.DataFrame(bus_data)

In [97]:
df_BS #display the data frame of BSRTC buses

,Route_Name,Route_link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,06h 40m,05:00,4.0,INR 550,54 Seats available
1,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),22:40,05h 20m,04:00,3.4,585,26 Seats available
2,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),20:28,05h 32m,02:00,1.9,630,6 Seats available
3,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,V.I.P Travels,A/C Seater / Sleeper (2+2),21:40,06h 00m,03:40,2.5,INR 350,51 Seats available
4,Patna (Bihar) to Motihari,https://www.redbus.in/bus-tickets/patna-to-mot...,Bihar state road transport corporation (BSRTC)...,Non AC Seater (2+2),21:20,03h 40m,01:00,3.1,INR 230,15 Seats available
...,...,...,...,...,...,...,...,...,...,...
102,Patna (Bihar) to Hazaribagh,https://www.redbus.in/bus-tickets/patna-to-haz...,MAA SHANTI TRAVELS,A/C Seater/Sleeper (2+1),21:25,06h 55m,04:20,4.5,INR 700,34 Seats available
103,Patna (Bihar) to Hazaribagh,https://www.redbus.in/bus-tickets/patna-to-haz...,Pappu Travels (yvts),A/C Seater/Sleeper (2+1),21:30,06h 00m,03:30,2.9,INR 800,10 Seats available
104,Patna (Bihar) to Hazaribagh,https://www.redbus.in/bus-tickets/patna-to-haz...,Tejas Maa Bhawani,A/C Seater / Sleeper (2+2),21:30,05h 00m,02:30,2.2,INR 800,4 Seats available
105,Patna (Bihar) to Hazaribagh,https://www.redbus.in/bus-tickets/patna-to-haz...,Tejas Maa Bhawani,A/C Sleeper (2+1),21:30,06h 30m,04:00,2.2,990,2 Seats available


In [99]:
df_BS .to_csv('BSRTC_bus_detais.csv',index=False) #converting dataframes to csv